In [1]:
from google.colab import drive
drive.mount("/content/drive")
!cp -r /content/drive/MyDrive/IIT-Projects/third-umpire-decision-automation/.creds/.aws ~/.aws
!pip install -qq sagemaker ultralytics onnx onnxruntime

Mounted at /content/drive


In [25]:
import boto3
s3_res = boto3.resource("s3")
s3_cli = boto3.client("s3")
bucket_name = "third-umpire-decision-automation-osura"
bucket = s3_res.Bucket(bucket_name)

In [5]:
tj_key = "training-jobs"
obj_keys = []
for obj in bucket.objects.filter(Prefix=tj_key):
  obj_keys.append(obj.key)

In [47]:
from datetime import datetime
import os, shutil
from ultralytics import YOLO

tasks = [
    "wicket-classification",
    "batsman-segmentation",
    "cricket-object-detect",
]
model_version = 2

for task in tasks:
  task_model_keys = [{"key":key} for key in obj_keys if task in key and key.endswith("output.tar.gz")]
  for key_det in task_model_keys:
    time_str = "-".join(key_det["key"].split("/")[-3].split("-")[2:-1])
    time = int(datetime.strptime(time_str, "%Y-%m-%d-%H-%M-%S").timestamp())
    key_det["time"] = time
  task_model_keys=sorted(task_model_keys, key=lambda key_det: -key_det["time"])

  for key_det in task_model_keys:
    src_key = key_det["key"]
    time = key_det["time"]
    dst_dir = f"./tmp/{task}/{time}"
    os.makedirs(dst_dir, exist_ok=True)
    archive_path = f"{dst_dir}/output.tar.gz"
    s3_cli.download_file(Bucket=bucket_name, Key=src_key, Filename=archive_path)
    shutil.unpack_archive(archive_path, dst_dir)
    src_weights_path = f"{dst_dir}/{task}/weights/best.pt"
    if os.path.exists(src_weights_path):
      model = YOLO(src_weights_path)
      onnx_src_path = model.export(format='onnx')
      weights_name = f"yolov8_{task.replace('-','_')}-v{model_version}.onnx"
      dst_key = f"model-weights/{weights_name}"
      s3_cli.upload_file(Filename=onnx_src_path, Bucket=bucket_name, Key=dst_key)
      break
  break
  

Ultralytics YOLOv8.0.105 🚀 Python-3.10.11 torch-2.0.1+cu118 CPU
YOLOv8n-cls summary (fused): 73 layers, 1437442 parameters, 0 gradients

PyTorch: starting from tmp/wicket-classification/1684742571/wicket-classification/weights/best.pt with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 2) (2.8 MB)

ONNX: starting export with onnx 1.14.0 opset 17...
ONNX: export success ✅ 0.6s, saved as tmp/wicket-classification/1684742571/wicket-classification/weights/best.onnx (5.5 MB)

Export complete (1.0s)
Results saved to /content/tmp/wicket-classification/1684742571/wicket-classification/weights
Predict:         yolo predict task=classify model=tmp/wicket-classification/1684742571/wicket-classification/weights/best.onnx imgsz=224 
Validate:        yolo val task=classify model=tmp/wicket-classification/1684742571/wicket-classification/weights/best.onnx imgsz=224 data=/opt/ml/input/data/train 
Visualize:       https://netron.app


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

